In [21]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import optuna
from tqdm import tqdm
from py.utilities import train_model
from optuna.samplers import TPESampler
import os
from optuna.study import StudyDirection
from math import sqrt

In [7]:
class PixelDataset(Dataset):

    def __init__(self, mode):
        #data loading in base alla modalità
        if mode == 'train':
            data = pd.read_csv(r'..\pixelwise\dataset\px_train_set.csv', header=0)
            scaler = StandardScaler()
            scaler.fit(data.drop('SWP', axis=1))
            joblib.dump(scaler, r'..\pixelwise\dataset\scaler.pkl')
        elif mode == 'val':
            data = pd.read_csv(r'..\pixelwise\dataset\val_set.csv', header=0)    
        elif mode == 'test':
            data = pd.read_csv(r'..\pixelwise\dataset\test_set.csv', header=0)   
        else:
            raise ValueError('Invalid mode')

        self.X = data.drop('SWP', axis=1).reset_index(drop=True) # features
        self.y = data['SWP'] # lista di float
        self.scaler = joblib.load(r'..\pixelwise\dataset\scaler_pixelwise.pkl')
        self.X = self.scaler.transform(self.X) # scaling features con scaler addestrato sul training set
        self.n_samples = len(self.y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [8]:
class PixelNet(nn.Module):

    def __init__(self, in_count, num_hidden_layers, size_hidden_layers, dropout_rates, device):
        super(PixelNet, self).__init__()

        self.num_hidden_layers = num_hidden_layers
        self.size_hidden_layers = size_hidden_layers
        self.dropout_rates = dropout_rates
        self.device = device

        #MLP for regression
        # Input layer
        self.input_layer = nn.Linear(in_count, size_hidden_layers[0])

        # Hidden layers
        self.hidden_layers = nn.ModuleList()
        for i in range(num_hidden_layers - 1):  # Loop per ogni hidden layer
            self.hidden_layers.append(nn.Linear(size_hidden_layers[i], size_hidden_layers[i+1]))
        
        # Output layer
        self.output_layer = nn.Linear(size_hidden_layers[-1], 1)# regression task
        # Dropout layers
        self.dropouts = nn.ModuleList([nn.Dropout(p=dropout_rates[i]) for i in range(len(dropout_rates))])

    def forward(self, x: Tensor):
        x = F.relu(self.input_layer(x))
        for i in range(self.hidden_layers):
            x = F.relu(self.hidden_layers[i](x))
            x = self.dropouts[i](x)
        x = self.output_layer(x)
        return x

In [24]:
loss_lists= {}
def objective(trial, batch_size, n_epochs, device, seed):

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    trial.set_user_attr("n_epochs", n_epochs)
    trial.set_user_attr("batch_size", batch_size)

    try:
        #suggerimento iperparametri
        num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 10)
        size_hidden_layers = [trial.suggest_int(f'neurons_per_layer_{i}', 16, 1024) for i in range(num_hidden_layers)] 
        learning_rate = trial.suggest_categorical('learning_rate', [1e-4, 5e-4, 1e-5, 5e-5, 1e-6, 5e-6])
        dropout_rates = [trial.suggest_float(f'dropout_rates_{i}', 0.1, 0.8) for i in range(num_hidden_layers)]
        num_epochs = n_epochs
        batch_size = batch_size
        
        # Crea datasets e dataloaders
        train_set = PixelDataset(mode='train')
        val_set = PixelDataset(mode='val')
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=batch_size)

        #creazione modello
        model = PixelNet(num_hidden_layers, size_hidden_layers, dropout_rates, device)
        model = model.to(device)

        #training
        best_val_loss, avg_train_losses, avg_val_losses, early_stop_epoch = train_model(model=model, 
                                                                                        num_epochs=num_epochs, 
                                                                                        lr=learning_rate, 
                                                                                        train_loader=train_loader, 
                                                                                        val_loader=val_loader,
                                                                                        device=device)
        

        #lista per plottare train/val loss della trial migliore
        loss_lists[trial.number] = (avg_train_losses, avg_val_losses, early_stop_epoch)

        return best_val_loss
    except Exception as e:
        print(f"An exception occurred during optimization: {e}")
        print("Saving study and exiting...")
        joblib.dump(study, r'..\pixelwise\optuna_results.pkl')
        raise

In [25]:
seed = 42
'''
parser = argparse.ArgumentParser(description='Ottimizzazione iperparametri con Optuna')
parser.add_argument('--batch_size', type=int, default=30, help='Dimensione del batch')
parser.add_argument('--n_trials', type=int, default=10, help='Numero di tentativi di ottimizzazione')
parser.add_argument('--n_epochs', type=int, default=200, help='Numero di epoche di addestramento')
args = parser.parse_args()  
'''
n_trials = 5
batch_size = 5094
n_epochs = 10

#usa una GPU se disponibile
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}") 

sampler = TPESampler(seed=42)

#Se lo studio non esiste, lo crea. Se esiste, lo carica e stampa la trial migliore finora.
if not os.path.exists(r'..pixelwise\optuna_results.pkl'):
    study = optuna.create_study(direction=StudyDirection.MINIMIZE, sampler=sampler)
else:
    study = joblib.load(r'..\pixelwise\optuna_results.pkl')
    print("Best trial until now:")
    print("  Number: ", study.best_trial.number)
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
    for trial in study.trials:
        if trial.state == optuna.trial.TrialState.FAIL: 
            study.enqueue_trial(trial.params)
            print(f"enqueued previosly failed trial...{trial}")

with tqdm(total=n_trials, desc="Optimizing.. ") as trial_pbar:
    def callback(study, trial):
        trial_pbar.set_description(f"Trial {trial.number+1}/{n_trials}")
        trial_pbar.update(1)
        joblib.dump(study, r'..\pixelwise\optuna_results.pkl')
    
    study.optimize(lambda trial: objective(trial, batch_size, n_epochs, device, seed),
                    n_trials=n_trials,
                    callbacks=[callback])

print(study.trials_dataframe())
print(f'Number of finished trials: {len(study.trials)}')
#print(f'Best trial: {study.best_trial}')
print(f'Best MSE: {study.best_trial.value}')
print(f'Best RMSE: {sqrt(study.best_trial.value)}')
print(f'Best value: {study.best_value}')
print(f'Best hyperparameters: {study.best_params}')

#plot_best_losses(loss_lists[study.best_trial.number])

[I 2024-05-23 11:37:11,143] A new study created in memory with name: no-name-2518d45f-4932-4ffe-b3a1-956ce79cf030


Using device: cpu


Optimizing.. :   0%|          | 0/5 [00:00<?, ?it/s][W 2024-05-23 11:37:11,153] Trial 0 failed with parameters: {'num_hidden_layers': 4, 'neurons_per_layer_0': 975, 'neurons_per_layer_1': 754, 'neurons_per_layer_2': 620, 'neurons_per_layer_3': 173, 'learning_rate': 1e-05, 'dropout_rates_0': 0.778936896513396, 'dropout_rates_1': 0.6827098485602953, 'dropout_rates_2': 0.24863737747479334, 'dropout_rates_3': 0.22727747704497045} because of the following error: FileNotFoundError(2, 'No such file or directory').
Traceback (most recent call last):
  File "C:\Users\giova\AppData\Local\Temp\ipykernel_9744\2245859744.py", line 19, in objective
    train_set = PixelDataset(mode='train')
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\giova\AppData\Local\Temp\ipykernel_9744\2090658673.py", line 6, in __init__
    data = pd.read_csv(r'..\data\pixelwise\px_train_set.csv', header=0)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\giova\OneDrive\De

An exception occurred during optimization: [Errno 2] No such file or directory: '..\\data\\pixelwise\\px_train_set.csv'
Saving study and exiting...


FileNotFoundError: [Errno 2] No such file or directory: '..\\pixelwise\\optuna_results.pkl'

In [27]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\giova\\OneDrive\\Desktop\\projects\\SWP-regr'

In [28]:
data_dir = os.path.join(current_dir, 'pixelwise', 'dataset')
data_dir

'c:\\Users\\giova\\OneDrive\\Desktop\\projects\\SWP-regr\\pixelwise\\data'

In [1]:
import joblib
study = joblib.load(r'C:\Users\CR-MASTER\Desktop\projects\SWP_Mapping\pixelwise\optuna_results_pixelwise.pkl')
df = study.trials_dataframe()
df.sort_values('value').head(20)

c:\Users\CR-MASTER\Desktop\projects\SWP_Mapping\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,number,value,datetime_start,datetime_complete,duration,params_dropout_rates_0,params_dropout_rates_1,params_dropout_rates_2,params_dropout_rates_3,params_dropout_rates_4,...,params_neurons_per_layer_4,params_neurons_per_layer_5,params_neurons_per_layer_6,params_neurons_per_layer_7,params_neurons_per_layer_8,params_neurons_per_layer_9,params_num_hidden_layers,user_attrs_batch_size,user_attrs_n_epochs,state
1293,1293,50.700161,2024-05-25 08:10:55.829223,2024-05-25 08:13:16.392621,0 days 00:02:20.563398,0.669408,0.126023,0.254247,0.300408,0.186443,...,968.0,926.0,711.0,566.0,925.0,NaN,9,5094,500,COMPLETE
1793,1793,50.980209,2024-05-25 18:38:45.796607,2024-05-25 18:41:07.067363,0 days 00:02:21.270756,0.614845,0.112141,0.222130,0.271946,0.545218,...,935.0,940.0,739.0,786.0,896.0,NaN,9,5094,500,COMPLETE
1191,1191,51.022938,2024-05-25 05:59:05.273400,2024-05-25 06:01:35.807861,0 days 00:02:30.534461,0.666553,0.141246,0.234048,0.336649,0.181022,...,943.0,881.0,736.0,575.0,1024.0,NaN,9,5094,500,COMPLETE
1175,1175,51.161748,2024-05-25 05:35:32.908779,2024-05-25 05:38:39.823266,0 days 00:03:06.914487,0.637057,0.161111,0.199473,0.352014,0.175301,...,906.0,842.0,635.0,510.0,970.0,NaN,9,5094,500,COMPLETE
1498,1498,51.182122,2024-05-25 12:38:09.602069,2024-05-25 12:40:33.805901,0 days 00:02:24.203832,0.380391,0.126270,0.249919,0.332239,0.197743,...,889.0,913.0,648.0,665.0,973.0,NaN,9,5094,500,COMPLETE
1318,1318,51.424708,2024-05-25 08:45:25.787863,2024-05-25 08:47:52.434606,0 days 00:02:26.646743,0.694661,0.136419,0.232931,0.298976,0.170619,...,939.0,831.0,757.0,661.0,954.0,NaN,9,5094,500,COMPLETE
1926,1926,51.549890,2024-05-25 21:30:51.560239,2024-05-25 21:34:05.758231,0 days 00:03:14.197992,0.726071,0.160445,0.238622,0.257934,0.571992,...,999.0,1007.0,686.0,493.0,912.0,NaN,9,5094,500,COMPLETE
1207,1207,51.768430,2024-05-25 06:17:58.006672,2024-05-25 06:20:27.713786,0 days 00:02:29.707114,0.679842,0.166583,0.215509,0.330294,0.166015,...,901.0,881.0,657.0,521.0,926.0,NaN,9,5094,500,COMPLETE
1798,1798,51.865879,2024-05-25 18:46:23.029955,2024-05-25 18:48:36.733548,0 days 00:02:13.703593,0.594927,0.131485,0.221280,0.266225,0.622847,...,936.0,974.0,755.0,800.0,1024.0,NaN,9,5094,500,COMPLETE
1646,1646,51.935519,2024-05-25 15:36:59.391652,2024-05-25 15:40:00.651623,0 days 00:03:01.259971,0.623943,0.143894,0.242002,0.320682,0.135890,...,939.0,975.0,740.0,755.0,1005.0,545.0,10,5094,500,COMPLETE


In [2]:
from math import sqrt
print(f"Best trial: {study.best_trial.number}")
print(f"    MSE: {study.best_trial.value}")
print(f"    RMSE: {sqrt(study.best_trial.value)}")
print(f"    params: {study.best_params}")

Best trial: 1293
    MSE: 50.700160971283914
    RMSE: 7.1204045510970735
    params: {'num_hidden_layers': 9, 'neurons_per_layer_0': 359, 'neurons_per_layer_1': 556, 'neurons_per_layer_2': 412, 'neurons_per_layer_3': 976, 'neurons_per_layer_4': 968, 'neurons_per_layer_5': 926, 'neurons_per_layer_6': 711, 'neurons_per_layer_7': 566, 'neurons_per_layer_8': 925, 'learning_rate': 0.0005, 'dropout_rates_0': 0.66940799302729, 'dropout_rates_1': 0.12602273793191196, 'dropout_rates_2': 0.25424703566336665, 'dropout_rates_3': 0.3004078921773323, 'dropout_rates_4': 0.186443002734407, 'dropout_rates_5': 0.47154497645078153, 'dropout_rates_6': 0.46653028984884215, 'dropout_rates_7': 0.324172442785381, 'dropout_rates_8': 0.30115830818835654}
